### Проміжні результати роботи над комп'ютерним проєктом

Виконала команда 12: Оксана Москв'як, Андрій Кульбаба, Олег Гуцуляк, Олександр Стаднік.

### Код Хаффмана (Huffman Coding)

Кодування Хафммана реалізоване за допомогою структури дерева Хаффмана як основи.

- створено клас об'єкта вершини дерева 
- створено рекурсивну функцію, яка генерує коди для кожного знаку
- створено функцію побудови дерева через об'єднання двох вершин з мінімальними значеннями в 1
- функція encoding - закодовує файл, наданий нам використовуючи при цьому mmap для роботи з файлами(для збереження пам'яті та підтримки різних форматів), будує дерево, словник значень, генерує коди для знаків, зберігає словник(json) знаків, кількість істиної інформації, початкове розширення файлу для декодування.
- функція decoding - розкодовує закодований нами файл відповідно до словника та початкового розширення.

Інші використані структури:
- Черга(deque) - для видалення переглянутого "декодованого" знаку
- defaultdict - у створенні словника, який зберігає кількість використання певного знаку
- bitarray - для збереження результату кодування
- heapq - використовується як структура min-heap - повне бінарне дерево, корінь якого має найменше значення серед усіх його нащадків, це має виконуватись для лівого і правого піддерев, потрібна щоб об'єднати дві вершини мінімального значення в 1 при побудові.

#### Приклад - "Місто", формат txt

початковий розмір файлу

In [1]:
import os

misto_size = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.txt').st_size
misto_size

911282

вбудований zip-архіватор

In [2]:
misto_size_built = os.stat('./compression/pidmohylnyy-valerian-petrovych-misto76.txt.zip').st_size
misto_size_built

277784

Кодування Гаффмана

In [4]:
misto_size_huffman = os.stat('./compressed_misto_huffman.bin').st_size
misto_size_huffman

304038

#### Приклад - великий json file

початковий розмір файлу

In [6]:
json_size = os.stat('./compression/large-file.json').st_size
json_size

26141343

вбудований zip-архіватор

In [7]:
big_json_size = os.stat('./compression/large-file.json.zip').st_size
big_json_size

3845106

Кодування Гаффмана

In [9]:
big_json_huffman_size = os.stat('./compressed_json_huffman.bin').st_size
big_json_huffman_size

17012930

#### Приклад - csv файл з користувачами на 100000 рядків

початковий розмір файлу

In [10]:
csv_size = os.stat('./compression/customers-100000.csv').st_size
csv_size

17317579

вбудований zip-архіватор

In [11]:
customers_100000_size = os.stat('./compression/customers-100000.csv.zip').st_size
customers_100000_size

8197738

Кодування Гаффмана

In [13]:
customers_100000_huffman_size = os.stat('compressed_customers_100000_huffman.bin').st_size
customers_100000_huffman_size

11956603

### LZ77

### LZ78

### LZW